In [ ]:
#! pip install pylibjpeg pylibjpeg-libjpeg pydicom
#! pip install -U python-gdcm

In [ ]:
#!python -m pip download python-gdcm -q
#!python -m pip download pylibjpeg-libjpeg -q
#!python -m pip download pylibjpeg -q

In [ ]:
import os
import sys
sys.path.append('../input/kerasapplications')
sys.path.append('../input/efficientnet-keras-source-code/')
sys.path.append('../input/python-gdcm/')
import keras_applications
import efficientnet.tfkeras as efficientnet

In [ ]:
!{sys.executable} -m pip install '../input/mypackage/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl' -q
!{sys.executable} -m pip install '../input/mypackage/pylibjpeg_libjpeg-1.3.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl' -q
!{sys.executable} -m pip install '../input/mypackage/python_gdcm-3.0.19-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl' -q
!{sys.executable} -m pip install '../input/mypackage/pylibjpeg-1.4.0-py3-none-any.whl' -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc

import gdcm
import pydicom as dicom
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.data import get_testdata_files

from path import Path
import scipy.ndimage
from tqdm import tqdm
import nibabel as nib
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob
import warnings
import dask.array as da

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator
from keras import layers

from PIL import Image as im
import traceback

import imageio


#from efficientnet_v2 import EfficientNetV2S
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

In [ ]:
bad = np.array([['1.2.826.0.1.3680043.10197_C1', '1.2.826.0.1.3680043.10197','C1'],['1.2.826.0.1.3680043.10454_C1', '1.2.826.0.1.3680043.10454','C1'],['1.2.826.0.1.3680043.10690_C1', '1.2.826.0.1.3680043.10690','C1']], dtype=np.object)

train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")

train_dir = '../input/rsna-2022-cervical-spine-fracture-detection/train_images'
test_dir = '../input/rsna-2022-cervical-spine-fracture-detection/test_images'
first_image = os.path.join(test_dir, test_df['StudyInstanceUID'].iloc[0])

new_submission = []
means = train_df.median(numeric_only=True).to_dict()
means = dict(zip(train_df.columns[1:], np.average(train_df[train_df.columns[1:]], axis=0, weights=train_df["patient_overall"] + 1)))
prediction_type = test_df['prediction_type'].tolist()
submission = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv')
for i in range(len(submission)):        
    new_submission.append(means[prediction_type[i]])
submission['fractured'] = new_submission


if(test_df.values[0][0] == bad[0][0]): test_df = pd.DataFrame({"row_id": ['1.2.826.0.1.3680043.22327_C1', '1.2.826.0.1.3680043.25399_C1', '1.2.826.0.1.3680043.5876_C1'], "StudyInstanceUID": ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876'], "prediction_type": ["C1", "C1", "C1"]})  
prediction_type_mapping = test_df['prediction_type'].map({'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6}).values

In [ ]:
#Loading Scans
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Load the scans in given folder path
def load_scan(path):
    
    dcm_paths = glob(f"{path}/*")
    dcm_paths.sort(key=natural_keys)
    
    patient_scan = [dicom.dcmread(paths) for paths in dcm_paths]
    
    return patient_scan

In [ ]:
def get_pixels_hu(slices):
   
    image = np.stack([cv2.resize(s.pixel_array,(256,256),interpolation = cv2.INTER_NEAREST) for s in slices])
    #image = np.stack([cv2.cvtColor(s.pixel_array.reshape(512,512),cv2.COLOR_GRAY2RGB) for s in slices])

    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16) #int16
    image = da.from_array(image) #Using Dask to speed up processing
    
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image <= -1000] = 0
    
    # Convert to Hounsfield units (HU)
        
    intercept = da.from_array([slices[slice_number].RescaleIntercept for slice_number in range(len(slices))])
    slope = da.from_array([slices[slice_number].RescaleSlope for slice_number in range(len(slices))])
    
    intercept=intercept.reshape((-1,1,1))
    slope=slope.reshape((-1,1,1))
    #print(slope.shape)
    #print(image.shape)
    image= slope * image.astype("float64")
    #print(image.shape)   
    image+= intercept
    #print(image.shape)      
    return image.astype("int16")

In [ ]:
MIN_BOUND = 150.0
MAX_BOUND = 2050.0
    
def normalize(image):
    image = (image - MIN_BOUND)*255.0 / (MAX_BOUND - MIN_BOUND)
    image[image>255] = 255.
    image[image<0] = 255.
    
    image = image.astype(np.int16)
    return image

In [ ]:
model_whole = keras.models.load_model('../input/effnet-whole-model-large/effnet_whole_128.h5')

In [ ]:
#model_classification = keras.models.load_model('../input/effnetclassificationmodel/Classifier_v3.h5')

#### Test Generator

- Yields only image samples

In [ ]:
def RSNATestGenerator(test_df, batch_size, infinite = True, base_path = test_dir):
    while 1:        
        testset=[]
        testidt=[]
        for i in (range(len(test_df))):        
            if type(test_df) is list: idt = test_df[i]
            else: idt = test_df['StudyInstanceUID'].iloc[i]
            path = os.path.join(base_path, idt)
            
            patient_scan=load_scan(path)
            patient_hu=get_pixels_hu(patient_scan)
            patient_hu_normalised=normalize(patient_hu)
            
            if os.path.exists(path):
                 for j in range(len(os.listdir(path))):
                    img = patient_hu_normalised[j]
                    img = np.array(img)
                    img=cv2.resize(img,(128, 128))
                    image=img_to_array(img)
                    image=image/255.0
                    testset+=[image]
                    testidt+=[idt]
                    if len(testset) == batch_size:                        
                        yield np.array(testset)
                        testset = []
        if len(testset) > 0: yield np.array(testset)
        if not infinite: break

In [ ]:
for train_idx, val_idx in StratifiedKFold(5).split(train_df, train_df['patient_overall']):
    i=1
    K.clear_session()
    model = model_whole
    
    try: # the best we can do at the moment..
        preds = model.predict_generator(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))
        
        new_preds = []
        for pred_idx in range(len(preds)):
            new_preds.append(preds[pred_idx][prediction_type_mapping[pred_idx]])
        # submission['fractured'] += preds[:, prediction_type_mapping] / 5
        submission['fractured'] += np.array(new_preds) / 5
        
    except: traceback.print_exc()    

In [ ]:
# fracture_threshold = 0.55

# preds = model.predict_generator(RSNATestGenerator(test_df, 
#                                                   min(len(test_df), 64), 
#                                                   infinite = False, 
#                                                   base_path = test_dir),
#                                 steps = max((len(test_df) // 64), 1))
        
# new_preds = []
# for i in preds:
#     new_pred_j = []
#     overall = 0
#     for j in i:
#         if j>fracture_threshold:
#             overall = 1
#         new_pred_j.append(j)
        
#     new_pred_j.append(overall)
    
#     new_preds.append(new_pred_j)


In [ ]:
# df_submission = pd.DataFrame(columns = ['row_id', 'fractured'])

# test_idt = []

# for i in (range(len(test_df))):        
#     if type(test_df) is list: idt = test_df[i]
#     else: idt = test_df['StudyInstanceUID'].iloc[i]
#     test_idt.append(idt)

# row_id = []
# fracture = []
# for i in range(len(test_idt)):
#     for j in range(8):
#         if j != 7:
#             row_id.append(f'{test_idt[i]}_C{j+1}')
#             fracture.append(new_preds[i][j])
            
#         if j ==7:
#             row_id.append(f'{test_idt[i]}_patient_overall')
#             fracture.append(new_preds[i][j])
# df_submission['row_id'] = row_id
# df_submission['fractured'] = fracture
# df_submission

#### Submission

In [ ]:
# submission = pd.DataFrame(columns=['row_id', 'fractured'])
# submission['row_id'] = test_df['row_id']
# submission['fractured'] =  submission['row_id'].map(df_submission.set_index('row_id')['fractured'])
# print(submission.head())
# submission.to_csv('submission.csv', index=False)

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)